# Execute a C code

In [1]:
%%writefile Hello.c
#include<stdio.h>
int main()
{
    printf("Hello world");
}

Writing Hello.c


In [2]:
!gcc Hello.c -o Hello

In [3]:
!./Hello

Hello world

# Thread ID
Each thread will have copies of numthread and thid

In [16]:
%%writefile ThID.c
#include<stdio.h>
#include<omp.h>
int main()
{
#pragma omp parallel
{
    printf("Hello world from OpenOMP\n");
    int numt=omp_get_num_threads();
    printf("Number of threads in my program is  %d\n",numt);
    int thid=omp_get_thread_num();
    printf("My thread ID is %d\n",thid);
}
}

Writing ThID.c


In [17]:
!gcc -fopenmp ThID.c -o ThID
!./ThID

Hello world from OpenOMP
Number of threads in my program is  4
My thread ID is 1
Hello world from OpenOMP
Number of threads in my program is  4
My thread ID is 2
Hello world from OpenOMP
Number of threads in my program is  4
My thread ID is 3
Hello world from OpenOMP
Number of threads in my program is  4
My thread ID is 0


In [18]:
%env OMP_NUM_THREADS=4

env: OMP_NUM_THREADS=4


Within one thread the statements are executed sequentially
Between threads, the order is not determined
Threads may interleaved. Because it is completely dynamic

In [19]:
%%writefile ShaPri.c
#include<stdio.h>
#include<omp.h>
int main()
{
  int sum=0;

#pragma omp parallel
{
    int thid=omp_get_thread_num();
    printf("My thread ID is %d\n",thid);
    sum=sum+1;
}
printf("Sum is %d\n",sum);
}

Writing ShaPri.c


In [20]:
!gcc -fopenmp ShaPri.c -o ShaPri


In [21]:
!./ShaPri

My thread ID is 0
My thread ID is 3
My thread ID is 2
My thread ID is 1
Sum is 4


In [22]:
%%writefile Pi1.c
#include<stdio.h>
#include<stdlib.h>
#include<omp.h>
static long num_steps = 100000000;
double step;
int main()
{
int i;
double x, pi, sum = 0.0;
double start_time, run_time;
step = 1/(double)num_steps;
start_time = omp_get_wtime();
for(i = 0; i < num_steps; i++)
{
x = (i + 0.5)*step;
sum += 4.0/(1.0 + x * x);
}
pi = step * sum;
run_time = omp_get_wtime() - start_time;
printf("\n Pi value = %f total time = %f ",pi,run_time);
return 0;
}

Writing Pi1.c


In [23]:
!gcc -fopenmp Pi1.c -o Pi1

In [24]:
!./Pi1


 Pi value = 3.141593 total time = 0.514958 

In [25]:
%%writefile paraloop.c
#include<stdio.h>
#include<omp.h>
int main()
{
int sum=0;
int i;
#pragma omp parallel private(i) shared(sum)
{

    printf("Hello world from OpenOMP\n");
    #pragma omp for
    for(int i=0;i<10;i++)
    {
      sum=sum+i;
      printf("iteration  is %d sum is %d\n",i,sum);
    }
}
printf("the final sum is%d",sum);
}

Writing paraloop.c


In [26]:
!gcc -fopenmp paraloop.c -o paraloop

In [27]:
%%writefile Paralloop.c
#include<stdio.h>
#include<omp.h>
int main()
{
  int sum=0;
#pragma omp parallel
{
   #pragma omp for
    for(int i=0;i<10;i++)
    {
      sum=sum+i;
      printf("iteration  is %d sum is %d\n",i,sum);
    }
}
printf("the sum is %d\n",sum);
}

Writing Paralloop.c


In [28]:
!gcc -fopenmp Paralloop.c -o Paralloop

In [29]:
!./Paralloop

iteration  is 0 sum is 0
iteration  is 1 sum is 1
iteration  is 2 sum is 9
iteration  is 8 sum is 17
iteration  is 9 sum is 26
iteration  is 3 sum is 29
iteration  is 4 sum is 33
iteration  is 5 sum is 38
iteration  is 6 sum is 7
iteration  is 7 sum is 45
the sum is 45


In [30]:
!./Paraloop

/bin/bash: line 1: ./Paraloop: No such file or directory


In [31]:
%%writefile PiPara.c
#include<stdio.h>
#include<stdlib.h>
#include<omp.h>
#define MIN(x,y) ((x<y)?x:y)
static long num_steps = 100000000;
double step;
int main()
{
int NThreads = 4;
double x, pi, sum = 0.0;
double start_time, run_time;
step = 1/(double)num_steps;
omp_set_num_threads(NThreads);
int chunk_size = num_steps/NThreads;
start_time = omp_get_wtime();
#pragma omp parallel
{
int tid = omp_get_thread_num();
int lower = tid * chunk_size;
int upper = (tid + 1) * chunk_size;
for(int i = lower; i < MIN(upper, num_steps); i++)
{
x = (i + 0.5)*step;
sum += 4.0/(1.0 + x * x);
}
}
pi = step * sum;
run_time = omp_get_wtime() - start_time;
printf("\n Pi value = %f total time = %f ",pi, run_time);
return 0;
}

Writing PiPara.c


In [32]:
!gcc -fopenmp PiPara.c -o PiPara

In [33]:
!./PiPara


 Pi value = 2.041535 total time = 1.450941 

In [34]:
%%writefile PiPara2.c
#include<stdio.h>
#include<stdlib.h>
#include<omp.h>
#define MIN(x,y) ((x<y)?x:y)
static long num_steps = 100000000;
double step;
int main()
{
int NThreads = 32,actual_NThreads;
double pi;
double sum[NThreads];
double start_time, run_time;
step = 1/(double)num_steps;
omp_set_num_threads(NThreads);
int chunk_size = num_steps/NThreads;
#pragma omp parallel
{
double x;
int tid = omp_get_thread_num();
int numThreads = omp_get_num_threads();
if(tid == 0)
{
actual_NThreads = numThreads;
}
sum[tid] = 0.0;
int lower = tid * chunk_size;
int upper = (tid + 1) * chunk_size;
for(int i = lower; i < MIN(upper,num_steps); i++)
{
x = (i + 0.5)*step;
sum[tid] += 4.0/(1.0 + x * x);
}
}
pi = 0.0;
for(int i = 0; i < actual_NThreads; i++)
{
pi += sum[i];
}
pi = step*pi;
printf("\n Pi value = %f total time = %f ",pi, run_time);
return 0;
}

Writing PiPara2.c


In [35]:
!gcc -fopenmp PiPara2.c -o PiPara2

In [36]:
!./PiPara2


 Pi value = 3.141593 total time = 0.000000 

In [37]:
%%writefile Add.c
#include <stdio.h>
#include <omp.h>

#define n 10

int main()
{
	double a[n];
	double sum=0;
	float startTime, endTime,execTime;
	int i;
	int omp_rank;

	startTime = omp_get_wtime();
	#pragma omp parallel private (i) \\shared (a)
	{
		#pragma omp for
		for(i=0;i<n;i++)
		{
			omp_rank = omp_get_thread_num();
			a[i] = i;  	// Use Random function and assign a[i]
			sum = sum + a[i];
			printf("The value of a[%d] = %lf and sum = %lf done by worker Thread ID = %d\n", i, a[i], sum, omp_rank);
		}

	}
	endTime = omp_get_wtime();

	execTime = endTime - startTime;
	printf("%f \n",execTime);
	printf("\n final sum =%lf",sum);
	return(0);
}




Writing Add.c


In [38]:
!gcc -fopenmp Add.c -o Add

Add.c: In function ‘main’:
Add.c:15:42: error: stray ‘\’ in program
   15 |         #pragma omp parallel private (i) \\shared (a)
      |                                          ^
Add.c:15:43: error: stray ‘\’ in program
   15 |         #pragma omp parallel private (i) \\shared (a)
      |                                           ^


In [39]:
!./Add


/bin/bash: line 1: ./Add: No such file or directory


In [40]:
%%writefile Add2.c
#include <stdio.h>
#include <omp.h>

#define n 10

int main()
{
	double a[n];
	double sum=0, psum;
	float startTime, endTime,execTime;
	int i;
	int omp_rank;

	startTime = omp_get_wtime();
	#pragma omp parallel private (i,psum) shared (a, sum)
	{
		psum=0;
		#pragma omp  for
		for(i=0;i<n;i++)
		{
			omp_rank = omp_get_thread_num();
			a[i] = i ;  	// Use Random function and assign a[i]
			psum = psum + a[i];
			printf("The value of a[%d] = %f and psum = %f done by worker Thread ID = %d\n", i, a[i], psum, omp_rank);
		}
		#pragma omp critical(dosum)
		{
			//omp_rank = omp_get_thread_num();
			sum = sum + psum;
			printf("Final Sum = %f\n", sum);
		}

	}
	endTime = omp_get_wtime();

	execTime = endTime - startTime;
	printf("%f \n",execTime);
	return(0);
}


Writing Add2.c


In [41]:
!gcc -fopenmp Add2.c -o Add2

In [42]:
!./Add2

The value of a[3] = 3.000000 and psum = 3.000000 done by worker Thread ID = 1
The value of a[4] = 4.000000 and psum = 7.000000 done by worker Thread ID = 1
The value of a[5] = 5.000000 and psum = 12.000000 done by worker Thread ID = 1
The value of a[6] = 6.000000 and psum = 6.000000 done by worker Thread ID = 2
The value of a[7] = 7.000000 and psum = 13.000000 done by worker Thread ID = 2
The value of a[8] = 8.000000 and psum = 8.000000 done by worker Thread ID = 3
The value of a[9] = 9.000000 and psum = 17.000000 done by worker Thread ID = 3
The value of a[0] = 0.000000 and psum = 0.000000 done by worker Thread ID = 0
The value of a[1] = 1.000000 and psum = 1.000000 done by worker Thread ID = 0
The value of a[2] = 2.000000 and psum = 3.000000 done by worker Thread ID = 0
Final Sum = 12.000000
Final Sum = 25.000000
Final Sum = 42.000000
Final Sum = 45.000000
0.000488 


In [43]:
%%writefile reduct.c
#include <stdio.h>
#include <omp.h>

#define n 10

int main()
{
	double a[n];
	double sum=0;
	float startTime, endTime,execTime;
	int i;
	int omp_rank;

	startTime = omp_get_wtime();
	#pragma omp parallel private (i) shared (a) reduction(+:sum)
	{
		#pragma omp for
		for(i=0;i<n;i++)
		{
			omp_rank = omp_get_thread_num();
			a[i] = i; //* 10.236;  	// Use Random function and assign a[i]
			sum = sum + a[i];
			printf("The value of a[%d] = %lf and sum = %lf done by worker Thread ID = %d\n", i, a[i], sum, omp_rank);
		}

	}
	endTime = omp_get_wtime();
	printf("\n Final Sum is %f\n", sum );
	execTime = endTime - startTime;
	printf("%f \n",execTime);
	return(0);
}


Writing reduct.c


In [44]:
!gcc -fopenmp reduct.c -o reduct

In [45]:
!./reduct

The value of a[0] = 0.000000 and sum = 0.000000 done by worker Thread ID = 0
The value of a[1] = 1.000000 and sum = 1.000000 done by worker Thread ID = 0
The value of a[2] = 2.000000 and sum = 3.000000 done by worker Thread ID = 0
The value of a[8] = 8.000000 and sum = 8.000000 done by worker Thread ID = 3
The value of a[9] = 9.000000 and sum = 17.000000 done by worker Thread ID = 3
The value of a[3] = 3.000000 and sum = 3.000000 done by worker Thread ID = 1
The value of a[4] = 4.000000 and sum = 7.000000 done by worker Thread ID = 1
The value of a[5] = 5.000000 and sum = 12.000000 done by worker Thread ID = 1
The value of a[6] = 6.000000 and sum = 6.000000 done by worker Thread ID = 2
The value of a[7] = 7.000000 and sum = 13.000000 done by worker Thread ID = 2

 Final Sum is 45.000000
0.002769 
